# 上週作業

In [ ]:
import os

os.chdir("../../")

In [ ]:
import json

from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate, ChatPromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings

from src.io.path_definition import get_project_dir


embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Ingredients: {input}\nOrigin: {output}",
)

with open(os.path.join(get_project_dir(), 'tutorial', 'Week-1', 'recipe_train.json'), 'r') as f:
    recipe_train = json.load(f)

examples = []

for recipe in recipe_train:
    examples.append({"input": " ".join(recipe['ingredients']),
                     "output": recipe['cuisine']})

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # The list of examples available to select from.
    examples,
    # The embedding class used to produce embeddings which are used to measure semantic similarity.
    embeddings,
    # The VectorStore class that is used to store the embeddings and do a similarity search over.
    FAISS,
    # The number of examples to produce.
    k=5,
)
similar_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Find the recipe origin based on the ingredients",
    suffix="Ingredients: {ingredients}\nOrigin:",
    input_variables=["ingredients"],
)

In [ ]:
similar_prompt

In [ ]:
with open(os.path.join(get_project_dir(), 'tutorial', 'Week-1', 'recipe_test.json'), 'r') as f:
    recipe_test = json.load(f)

existing_ingredients = recipe_test[99]['ingredients']

similar_prompt.invoke(", ".join(existing_ingredients))

In [ ]:
from langchain.chat_models import ChatOpenAI

from src.initialization import credential_init

credential_init()

model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                   model_name="gpt-4o-2024-05-13", temperature=0)

In [ ]:
chain = similar_prompt|model

In [ ]:
chain.invoke(", ".join(existing_ingredients))

# Remote server

### 1. Making a POST Request (發送 POST 請求):

- requests.post(...) sends an HTTP POST request to the specified URL.
- The URL "http://localhost:5000/openai/invoke" points to a local server running on port 5000, at the endpoint /openai/invoke.
- The json parameter is used to send a JSON payload with the request. In this case, the payload is {'input': "Where is Taiwan"}.
- requests.post(...) 發送一個 HTTP POST 請求到指定的 URL。
- URL "http://localhost:5000/openai/invoke" 指向一個本地服務器，該服務器在端口 5000 上運行，並且指向 /openai/invoke 端點。
- json 參數用於隨請求發送 JSON 負載。在這個例子中，負載是 {'input': "Where is Taiwan"}。

### 2. Response Handling (響應處理):

- The server processes the request and sends back a response.
- The response is stored in the response variable, which can then be inspected or used further in the code.
- 服務器處理請求並返回響應。
- 響應存儲在 response 變量中，之後可以檢查或在代碼中進一步使用。

In [ ]:
import requests

response = requests.post(
    "http://localhost:5000/openai/invoke",
    json={'input': "Where is Taiwan"}
)

In [ ]:
response.json()

# Use the remote model 

## Basic Usage

### 1. Creating an Instance of RemoteRunnable (創建 RemoteRunnable 的實例):

- This line creates an instance of RemoteRunnable and initializes it with the URL of the remote language model service. In this case, the service is running locally on http://localhost:5000/openai/.
- 這行代碼創建一個 RemoteRunnable 的實例，並用遠程語言模型服務的 URL 進行初始化。在這個例子中，服務在本地運行，URL 為 http://localhost:5000/openai/。

In [ ]:
from langserve import RemoteRunnable

llm = RemoteRunnable("http://localhost:5000/openai/")

### 2. Asynchronous Streaming of Responses (異步流式處理回應):

- llm.astream("Where is Taiwan?") sends the query "Where is Taiwan?" to the remote service and retrieves the response as a stream.
- async for msg in ... is used to handle the streaming responses asynchronously.
- print(msg.content, end="", flush=True) prints each message content received from the stream without adding a new line after each message, and flushes the output buffer to ensure the message is displayed immediately.
- llm.astream("Where is Taiwan?") 將查詢 "Where is Taiwan?" 發送到遠程服務，並以流的形式檢索回應。
- async for msg in ... 用於異步處理流式回應。
- print(msg.content, end="", flush=True) 打印每個從流中接收到的消息內容，不在每個消息後添加新行，並刷新輸出緩衝區以確保消息立即顯示。

In [ ]:
# Supports astream
async for msg in llm.astream("Where is Taiwan?"):
    print(msg.content, end="", flush=True)

In [ ]:
output = llm.invoke("Where is Taiwan?")

In [ ]:
output.content

## Make the external service a part of the chain

### 1. Comedian Chain (喜劇演員鏈)

- ChatPromptTemplate.from_messages(...) creates a prompt template where the system prompt instructs the model to either tell a joke or state a fact, and the human prompt provides the input.
- This template is then piped (|) to a language model (llm) to generate the comedian's response.
- ChatPromptTemplate.from_messages(...) 創建一個提示模板，其中系統提示指示模型要麼講一個笑話，要麼陳述一個不搞笑的事實，並且僅輸出一個。
- 然後將此模板通過管道（|）傳遞給語言模型（llm），以生成喜劇演員的回應。

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

comedian_chain = (
    ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a comedian that sometimes tells funny jokes and other times you just state facts that are not funny. Please either tell a joke or state fact now but only output one.",
            ),
            ('human', '{input}'
            )
        ]
    )
    | llm
)


### 2. Joke Classifier Chain

- This chain is similar to the comedian chain but serves a different purpose.
- The system prompt asks the model to classify the joke as "funny" or "not funny" and repeat the first five words for reference.
- This template is also piped to the language model (llm).
- 這個鏈與喜劇演員鏈類似，但用途不同。
- 系統提示要求模型將笑話分類為“搞笑”或“不搞笑”，並重複笑話的前五個詞以供參考。
- 此模板也通過管道傳遞給語言模型（llm）。

In [ ]:
joke_classifier_chain = (
    ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Please determine if the joke is funny. Say `funny` if it's funny and `not funny` if not funny. Then repeat the first five words of the joke for reference...",
            ),
            ("human", "{joke}"),
        ]
    )
    | llm
)


### 3. Combining Chains with RunnablePassthrough

- This combines the comedian chain and the joke classifier chain using RunnablePassthrough.assign.
- The comedian chain generates the output, and then this output is passed to the joke classifier chain to classify its humor.
- 這將喜劇演員鏈和笑話分類器鏈結合在一起，使用 RunnablePassthrough.assign。
- 喜劇演員鏈生成輸出，然後將此輸出傳遞給笑話分類器鏈以分類其幽默性。

In [ ]:
chain = {"joke": comedian_chain} | RunnablePassthrough.assign(
    classification=joke_classifier_chain
)

In [ ]:
chain.invoke({"input": "A man and a beer"})

# ChatBot

In [ ]:
from IPython.display import Image

Image(url="https://python.langchain.com/v0.1/assets/images/chat_use_case-eb8a4883931d726e9f23628a0d22e315.png")

- N-Shot
- The historical chat history can be consdiered as a list of question-answer pairs
- If the chatbot doesn’t remember past chats, it’s called stateless because it doesn’t know what happened before.

## Basic Example

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

model.invoke(
    [
        HumanMessage(
            content="Translate this sentence from English to French: I love programming."
        )
    ]
)

In [ ]:
model.invoke([HumanMessage(content="What did you just say?")])

In [ ]:
model.invoke(
    [
        HumanMessage(
            content="Translate this sentence from English to French: I love programming."
        ),
        AIMessage(content="J'adore la programmation."),
        HumanMessage(content="What did you just say?"),
    ]
)

## Prompt templates

### 1. Creating a ChatPromptTemplate 

- ChatPromptTemplate.from_messages(...) creates a prompt template for the chatbot.
- The first message in the template is a system message: "You are a helpful assistant. Answer all questions to the best of your ability." This message sets the context and behavior of the assistant, instructing it to be helpful and thorough in its responses.
- MessagesPlaceholder(variable_name="messages") is a placeholder for dynamic content. The variable_name="messages" specifies that this placeholder will be filled with user messages during the conversation.
- ChatPromptTemplate.from_messages(...) 創建了一個聊天機器人的提示模板。
- 模板中的第一條消息是一條系統消息：“You are a helpful assistant. Answer all questions to the best of your ability.” 此消息設置了助手的上下文和行為，指示其在回答中要提供幫助並盡力而為。
- MessagesPlaceholder(variable_name="messages") 是一個動態內容的佔位符。variable_name="messages" 指定該佔位符將在對話中插入用戶消息。

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

### 2. Creating the Chain

- This line pipes (|) the prompt template to a language model (model).
- chain represents a sequence of operations where the prompt template is used to format user messages, and the language model processes these messages to generate responses.
- 這行代碼通過管道（|）將提示模板傳遞給語言模型（model）。
- chain 代表一系列操作，其中提示模板用於格式化用戶消息，語言模型處理這些消息以生成回應。

In [ ]:
chain = prompt | model

The MessagesPlaceholder above inserts chat messages passed into the chain's input as chat_history directly into the prompt. Then, we can invoke the chain like this:

In [ ]:
chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="Translate this sentence from English to French: I love programming."
            ),
            AIMessage(content="J'adore la programmation."),
            HumanMessage(content="What did you just say?"),
        ],
    }
)

## Example of Using MessageHistory

As a shortcut for managing the chat history, we can use a MessageHistory class, which is responsible for saving and loading chat messages. There are many built-in message history integrations that persist messages to a variety of databases, but for this quickstart we'll use a in-memory, demo message history called ChatMessageHistory

### 1. Importing the ChatMessageHistory Class (導入 ChatMessageHistory 類)

- This line imports the ChatMessageHistory class from the langchain.memory module. This class is used to handle the chat messages in memory.
- 這行代碼從 langchain.memory 模塊中導入 ChatMessageHistory 類。此類用於在內存中處理聊天消息。

In [ ]:
from langchain.memory import ChatMessageHistory

### 2. Creating an Instance of ChatMessageHistory (創建 ChatMessageHistory 的實例)

- This line creates an instance of ChatMessageHistory. This instance will store the chat messages in memory for this session.
- 這行代碼創建一個 ChatMessageHistory 的實例。該實例將在此會話期間將聊天消息存儲在內存中。

In [ ]:
demo_chat_history = ChatMessageHistory()

### 3. Adding User and AI Messages (添加用戶和 AI 消息)

- demo_chat_history.add_user_message("hi!") adds a user message ("hi!") to the chat history.
- demo_chat_history.add_ai_message("whats up?") adds an AI response ("whats up?") to the chat history.
- demo_chat_history.add_user_message("hi!") 將用戶消息（“hi!”）添加到聊天記錄中。
- demo_chat_history.add_ai_message("whats up?") 將 AI 回應（“whats up?”）添加到聊天記錄中。

In [ ]:
demo_chat_history.add_user_message("hi!")

demo_chat_history.add_ai_message("whats up?")

demo_chat_history.messages

### 4. Retrieving the Messages (檢索消息)

- This line retrieves the list of messages stored in demo_chat_history. Each message is an object that contains information about the sender (user or AI) and the content of the message.
- 這行代碼檢索存儲在 demo_chat_history 中的消息列表。每條消息都是一個對象，包含有關發送者（用戶或 AI）和消息內容的信息。

In [ ]:
{"messages": demo_chat_history.messages}

In [ ]:
demo_chat_history.add_user_message(
    "Translate this sentence from English to French: I love programming."
)

response = chain.invoke({"messages": demo_chat_history.messages})

response

In [ ]:
# Put the response back into the demo_chat_history

demo_chat_history.add_ai_message(response)

demo_chat_history.add_user_message("What did you just say?")

chain.invoke({"messages": demo_chat_history.messages})

# **** 預計第一個小時結束 ****

## Conversational Retrievers

- 土味情話反殺大全 (推薦上Youtube看)

In [ ]:
import pandas as pd
from langchain_community.vectorstores import Chroma
from langchain.docstore.document import Document

df = pd.DataFrame(data=[["确认过眼神，你是我爱的人。", "确认过眼神，我是你泡不到的人。"],
                         ["万水千山总是情，爱我多一点行不行。", "一寸光阴一寸金，劝你死了这条心。"],
                         ["今天吃了泡面，吃了炒面，还是想走进你的心里面。", "吃那么多面，最后还不是变成大便。"],
                         ["草莓，蓝莓，蔓越莓，今天你想我了没？", "冬瓜，西瓜，哈密瓜，你再巴巴我打得你叫妈妈。"],
                         ["众生皆苦，唯你独甜。", "尝遍众生，你为渣男代言。"],
                         ["你喜欢瑞士名表还是我帅气的外表？", "我喜欢去年买了个表。"],
                         ["我想问一条路，到哥哥心里的路。", "山路十八弯，走完脑血栓。"],
                         ["小姐姐，我心里给你留了一块地，死心塌地。", "对不起，我的心里只容得下一块地，玛莎拉蒂。"],
                         ["小姐姐你笑起来真好看啊。", "你看起来真好笑啊。"],
                         ["亲爱的你知道吗，你的笑容没有酒，我却醉得像条狗", "我的笑容没有酒，你是真的像条狗"],
                         ["宝贝儿，我在手上划了一道口子，你也划一下吧，这样我们就是两口子了", "我怕我们的血溶到一起，被你发现其实我是你爸爸"],
                         ["这世间万物都有尽头，落叶归根，而我归你", "对不起 我不收垃圾"],
                         ["请问……我想问一下路，那条通往你心里的路", "八格牙路"],
                         ["你今天怎么怪怪的？ 怪可爱的",  "你今天也怪怪的，怪恶心的"],
                         ["亲爱的，你知道我和唐僧的区别吗？ 唐僧取经我娶你", "知道你和沙僧的区别吗？ 他叫沙僧你叫沙雕"],
                         ["亲爱的，你不觉得累吗？ 你已经在我的脑海里跑了好几圈了", "傻孩子，我在找出口呢"],
                         ["莫文蔚的阴天。孙燕姿的雨天，周杰伦的晴天，都不如你和我聊天", "求求你了，能否还我一个宁静的夏天"],
                         ["如果你是方便面，那我就是白开水，今生今世，我泡定你了", "故事的最后，她变成了屎，你变成了尿，你们终究分道扬镳"],
                         ["大年三十晚上的鞭炮再响，也没有我想你那么想", "大年三十晚上的鞭炮再响，也没有你放的屁响"],
                         ["c罗可以上演帽子戏法，可我想你却没有办法", "c罗可以上演帽子戏法，我也可以给你上演绿帽子戏法"],
                         ["不要抱怨，抱我", "抱不起来，太重"],
                         ["你有没有发现我的眼睛很好看？因为我满眼都是你啊", "对不起，你眼睛在哪呢？"]], 
                  columns=['input', 'output'])


documents = []

for _, row in df.iterrows():
    documents.append(Document(page_content=row['input'], metadata={'output': row['output']}))

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = FAISS.from_documents(documents=documents, embedding=embeddings)

### 1. Define a Chat Prompt Template

- ChatPromptTemplate.from_messages is used to create a prompt template from a list of message tuples.
- The system message sets the context for the AI assistant, instructing it to act grumpy and respond with cheesy pickup lines in Simplified Chinese (簡體中文).
- MessagesPlaceholder(variable_name="messages") is a placeholder for the dynamic messages that will be inserted when the chain is invoked.
- The AI's response will be prefixed with ": " to indicate its reply.
- 使用 ChatPromptTemplate.from_messages 從訊息元組列表創建提示模板。
- 系統訊息設定了 AI 助手的上下文，指示它表現得很煩躁並用簡體中文 (簡體中文) 來回應俏皮的搭訕台詞。
- MessagesPlaceholder(variable_name="messages") 是動態訊息的佔位符，這些訊息會在調用管道時插入。
- AI 的回應將以 ": " 作為前綴，以表示其回應。

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", """
                      You are a helpful AI assistant acting as if you hava rough day and you are now very grumpy. 
                      You will respond with the following style, cheesy pickup lines, shown in the context:\n\n{context}

                      You will reply in simplified Chinese (簡體中文).
                      """,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)


### 2. Combine the Retrieval Chain and Prompt Template

- chat_chain combines the previously defined retrieval_chain with the prompt, creating a pipeline that processes input messages and generates responses based on the specified style and language.
- chat_chain 結合先前定義的 retrieval_chain 和 prompt，創建一個處理輸入訊息並基於指定風格和語言生成回應的管道。

In [ ]:
chain = prompt | model

In [ ]:
retriever = vectorstore.as_retriever(k=3)

docs = retriever.invoke("""我血糖低。你快跟我说几句甜蜜""")

docs_input = set()

for doc in docs:
    docs_input.add(f"Human:{doc.page_content}; AI:{doc.metadata['output']}")

In [ ]:
demo_chat_history = ChatMessageHistory()
demo_chat_history.add_user_message("""我血糖低。你快跟我说几句甜蜜""")

chain.invoke({"context": list(docs_input), "messages": demo_chat_history.messages})

## Conversational Retrievers + LCEL

In [ ]:
from typing import Dict

from langchain_core.runnables import RunnablePassthrough


def parse_retriever_input(params: Dict) -> str:
    
    """
    Extracts the content of the last message from the 'messages' list in the input dictionary.

    Args:
        params (Dict): A dictionary containing a key 'messages' which maps to a list of message objects. 
                       Each message object should have a 'content' attribute.

    Returns:
        str: The content of the last message in the 'messages' list.

    Raises:
        KeyError: If the 'messages' key is not found in the input dictionary.
        IndexError: If the 'messages' list is empty.
    """
    
    return params["messages"][-1].content


def context_build(docs) -> str:
    
    """
    Builds a list of strings formatted with human and AI content from the input documents.

    Args:
        docs (List[Dict]): A list of dictionaries, where each dictionary represents a document with 
                           'page_content' and 'metadata' keys. The 'metadata' key should contain another 
                           dictionary with an 'output' key.

    Returns:
        List[str]: A list of formatted strings, each combining the 'page_content' and 'output' metadata 
                   of a document in the form "human:{page_content}; ai:{output}".

    Example:
        If docs is:
        [
            {
                "page_content": "Document 1 content",
                "metadata": {"output": "Response 1"}
            },
            {
                "page_content": "Document 2 content",
                "metadata": {"output": "Response 2"}
            }
        ]

        The function will return:
        [
            "human:Document 1 content; ai:Response 1",
            "human:Document 2 content; ai:Response 2"
        ]

    Raises:
        KeyError: If the 'page_content' or 'metadata' keys are not found in a document.
        KeyError: If the 'output' key is not found in the 'metadata' dictionary.
    """
    
    docs_input = set()

    for doc in docs:
        docs_input.add(f"human:{doc.page_content}; ai:{doc.metadata['output']}")

    return list(docs_input)


### 1. Define a Retrieval Chain (定義檢索鏈):

- The retrieval_chain is created by chaining together several functions: parse_retriever_input, retriever, and context_build.
- RunnablePassthrough.assign is used to define a chain where the context is processed sequentially by these functions.
- retrieval_chain 是通過將幾個函數鏈接在一起創建的：parse_retriever_input，retriever 和 context_build。
- 使用 RunnablePassthrough.assign 來定義一個鏈，該鏈中的上下文依次由這些函數處理。

In [ ]:
retrieval_chain = RunnablePassthrough.assign(context=parse_retriever_input | retriever | context_build)

### 2. Create a Chat Message History:

- demo_chat_history is an instance of ChatMessageHistory, which stores the history of chat messages.
- add_user_message is used to add a user message to the chat history. The message here is "我血糖低。你快跟我说几句甜蜜", which translates to "My blood sugar is low. Quickly tell me something sweet."
- demo_chat_history 是 ChatMessageHistory 的實例，用於存儲聊天訊息歷史。
- add_user_message 用於將用戶訊息添加到聊天歷史中。此處的訊息是 "我血糖低。你快跟我說幾句甜蜜"，意思是 "我的血糖低了。快跟我說些甜言蜜語。"

In [ ]:
demo_chat_history = ChatMessageHistory()
demo_chat_history.add_user_message("""我血糖低。你快跟我说几句甜蜜""")

### 3. Invoke the Retrieval Chain

- retrieval_chain.invoke is called with the chat history messages as input. This processes the messages through the defined chain to extract relevant information or context.
- 使用聊天歷史訊息作為輸入調用 retrieval_chain.invoke。這會通過定義的鏈來處理訊息，從而提取相關信息或上下文。

In [ ]:
retrieval_chain.invoke({"messages": demo_chat_history.messages})

### 4. Combine the Retrieval Chain and Final Prompt Template

- Define a Chat Prompt Template (定義聊天提示模板)
- Combine the Retrieval Chain and Prompt Template (結合檢索鏈和提示模板)

In [ ]:
# Step 1: Define the chat prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", """
                      You are a helpful AI assistant acting as if you hava rough day and you are now very grumpy. 
                      You will respond with the following style, cheesy pickup lines, shown in the context:\n\n{context}

                      You will reply in simplified Chinese (簡體中文).
                      """,
        ),
        MessagesPlaceholder(variable_name="messages"),
        ("ai", ": ")
    ]
)

# Step 2: Combine the retrieval chain and the prompt template
chat_chain = retrieval_chain|prompt

In [ ]:
chat_chain.invoke({"messages": demo_chat_history.messages})

In [ ]:
chat_chain = retrieval_chain|prompt|model

response = chat_chain.invoke({"messages": demo_chat_history.messages})

In [ ]:
response

Put the response into the demo_chat_history

In [ ]:
demo_chat_history.add_ai_message(response.content)

In [ ]:
demo_chat_history

### 回家作業 2: 將retriever抽換成WikipediaRetriever

基本上，你可以將這個retriever的內容抽換成任何你需要的資料，來加快寫報告的效率。

## Compress the chat history to reduce the size of the prompt


https://github.com/langchain-ai/langserve/blob/main/examples/conversational_retrieval_chain/server.py

In [ ]:
from langchain.memory import ChatMessageHistory

demo_chat_history = ChatMessageHistory()

demo_chat_history.add_user_message("hi!")

demo_chat_history.add_ai_message("whats up?")

demo_chat_history.messages

In [ ]:
message = demo_chat_history.messages[0]

In [ ]:
message.content

In [ ]:
message.type

In [ ]:
from langchain_core.runnables import RunnableMap, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


def _format_chat_history(chat_history: ChatMessageHistory) -> str:
    """
    Format chat history into a string representation.

    Args:
        chat_history (ChatMessageHistory): An instance of ChatMessageHistory containing
                                           messages to be formatted.

    Returns:
        str: A formatted string representing the chat history. Each message is formatted as
             "{type}: {content}", where 'type' is the message type ('user' or 'ai') and
             'content' is the message content. Messages from the AI are separated by an 
             additional newline for readability.

    Example:
        If chat_history contains messages:
        [
            {'type': 'user', 'content': 'Hello!'},
            {'type': 'ai', 'content': 'Hi there! How can I help?'},
            {'type': 'user', 'content': 'I need assistance with a problem.'},
            {'type': 'ai', 'content': 'Sure, I'm here to help.'}
        ]

        The function will return:
        "user: Hello! \nai: Hi there! How can I help? \n\nuser: I need assistance with a problem. \nai: Sure, I'm here to help. \n\n"

    Notes:
        - Messages from the AI ('ai' type) are separated by an additional newline to distinguish 
          between consecutive AI responses.
    """
    buffer = "\n"
    for message in chat_history.messages:
        type = message.type
        content = message.content
        buffer += f"{type}: {content} \n"
        if type == 'ai':
            buffer += "\n"
    
    return buffer


### 1. Define a template for the prompt

- _TEMPLATE is a string template designed to rephrase a follow-up question into a standalone question using the provided chat history.
- The template specifies how the chat history and the follow-up question should be formatted in the prompt.
- _TEMPLATE 是一個字串模板，旨在使用提供的聊天歷史將後續問題重新表述為獨立問題。
- 該模板指定了聊天歷史和後續問題在提示中的格式。

In [ ]:
_TEMPLATE = """Given the following conversation and a follow up question, rephrase the 
follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:
"""

### 2. Create a PromptTemplate Instance

- CONDENSE_QUESTION_PROMPT is an instance of PromptTemplate created from _TEMPLATE.
- This instance is used to generate prompts that will guide the model in rephrasing the follow-up question.\
- CONDENSE_QUESTION_PROMPT 是從 _TEMPLATE 創建的 PromptTemplate 實例。
- 此實例用於生成提示，指導模型重新表述後續問題。

In [ ]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_TEMPLATE)

### 3. Define Input Processing Workflow

- _inputs is a RunnableMap instance that defines the workflow for processing inputs to generate the standalone question.
- The workflow includes:
    - Formatting Chat History: Using a lambda function to format the chat history with _format_chat_history.
    - Generating the Prompt: Using CONDENSE_QUESTION_PROMPT to create the prompt with the formatted chat history and follow-up question.
    - Running the Model: Passing the prompt through the model to get the standalone question.
    - Parsing the Output: Using StrOutputParser to convert the model's output into a string.
    
- _inputs 是一個 RunnableMap 實例，定義了處理輸入以生成獨立問題的工作流程。
- 工作流程包括：
    - 格式化聊天歷史： 使用 lambda 函數和 _format_chat_history 來格式化聊天歷史。
    - 生成提示： 使用 CONDENSE_QUESTION_PROMPT 用格式化的聊天歷史和後續問題來創建提示。
    - 運行模型： 通過模型傳遞提示以獲取獨立問題。
    - 解析輸出： 使用 StrOutputParser 將模型的輸出轉換為字串。

In [ ]:
_inputs = RunnableMap(
    standalone_question=RunnablePassthrough.assign(
    chat_history=lambda x: _format_chat_history(x["chat_history"])
    )
    | CONDENSE_QUESTION_PROMPT
    | model
    | StrOutputParser(),
)

No over yet, we just finished the `QUESTION` generation part.

In [ ]:
from IPython.display import Image

Image("tutorial/Week-4/_inputs_pipeline.png")

In [ ]:
_inputs.invoke({"question": "Translate this sentence from English to French: I love programming.",
                "chat_history": demo_chat_history})

In [ ]:
print(_format_chat_history(demo_chat_history))

### Question - Retrieval Chain

- This code snippet demonstrates how to build a conversational question-answering (QA) chain using LangChain. The chain processes input data to generate a standalone question, retrieves relevant context, and finally generates an answer based on that context.

In [ ]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate, PromptTemplate, format_document


DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")


def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    """Combine documents into a single string."""
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

### 1. Define Context Mapping

- _context is a dictionary that maps the input processing steps to extract and combine relevant information:
    - "context": This key uses a pipeline (itemgetter("standalone_question") | retriever | _combine_documents) to extract the standalone question, retrieve relevant documents, and combine them into a single context.
    - "question": This key uses a lambda function to directly map the standalone question from the input data.
    
- _context 是一個字典，用於映射輸入處理步驟以提取和組合相關信息：
    - "context": 此鍵使用一個管道（itemgetter("standalone_question") | retriever | _combine_documents）來提取獨立問題，檢索相關文檔，並將它們組合成單一上下文。
    - "question": 此鍵使用一個 lambda 函數來直接映射輸入數據中的獨立問題。

In [ ]:
_context = {"context": itemgetter("standalone_question")|retriever|_combine_documents, 
            "question": lambda x: x["standalone_question"]}

### 2. Create an Answer Prompt Template

- ANSWER_PROMPT is an instance of ChatPromptTemplate created from a template. This template guides the model to answer the question based only on the provided context.
- ANSWER_PROMPT 是從模板創建的 ChatPromptTemplate 實例。此模板指導模型僅基於提供的上下文來回答問題。

In [ ]:
ANSWER_PROMPT = ChatPromptTemplate.from_template(
    """Answer the question based only on the following context:
       {context}
       Question: {question}
    """)

### 3. Define Conversational QA Chain

- conversational_qa_chain is a pipeline that processes the input data through several stages:
    - _inputs: Processes input data to generate a standalone question.
    - _context: Extracts and combines the context needed to answer the question.
    - ANSWER_PROMPT: Generates a prompt for the model using the context and question.
    - model: Runs the model to generate an answer.
    - StrOutputParser: Parses the model's output into a string.
    
- conversational_qa_chain 是一個處理輸入數據的管道，通過幾個階段生成答案：
    - _inputs: 處理輸入數據以生成獨立問題。
    - _context: 提取並組合回答問題所需的上下文。
    - ANSWER_PROMPT: 使用上下文和問題生成提示。
    - model: 運行模型生成答案。
    - StrOutputParser: 將模型的輸出解析為字符串。

In [ ]:
conversational_qa_chain = (
    _inputs | _context | ANSWER_PROMPT | model | StrOutputParser()
)

In [ ]:
Image("tutorial/Week-4/conversation_qa_chain.png")

### Reverse Engineering

In [ ]:
_inputs = RunnableMap(
    standalone_question=RunnablePassthrough.assign(
    chat_history=lambda x: _format_chat_history(x["chat_history"])
    )
    | CONDENSE_QUESTION_PROMPT
)

_inputs.invoke({"question": "Translate this sentence from English to French: I love programming.",
                "chat_history": demo_chat_history})


In [ ]:
from operator import itemgetter

_inputs = RunnableMap(
    standalone_question=RunnablePassthrough.assign(
    chat_history=lambda x: _format_chat_history(x["chat_history"])
    )
    | CONDENSE_QUESTION_PROMPT
    | model
    | StrOutputParser(),
)

_context = {"context": itemgetter("standalone_question"),
            "question": lambda x: x["standalone_question"]}

conversational_qa_chain = (
    _inputs | _context 
)

conversational_qa_chain.invoke({"question": "Translate this sentence from English to French: I love programming.",
                                "chat_history": demo_chat_history})

In [ ]:
from operator import itemgetter

_inputs = RunnableMap(
    standalone_question=RunnablePassthrough.assign(
    chat_history=lambda x: _format_chat_history(x["chat_history"])
    )
    | CONDENSE_QUESTION_PROMPT
    | model
    | StrOutputParser(),
)

_context = RunnablePassthrough.assign(context=itemgetter('standalone_question')|retriever,
                                      question=itemgetter('standalone_question'))

# _context = {"context": itemgetter("standalone_question")|retriever|_combine_documents,
#             "question": lambda x: x["standalone_question"]}

"""
RunnablePassthrough.assign(
    chat_history=lambda x: _format_chat_history(x["chat_history"])
    )
"""

conversational_qa_chain = (
    _inputs | _context
)

conversational_qa_chain.invoke({"question": "Translate this sentence from English to French: I love programming.",
                                "chat_history": demo_chat_history})

In [ ]:
from operator import itemgetter

_inputs = RunnableMap(
    standalone_question=RunnablePassthrough.assign(
    chat_history=lambda x: _format_chat_history(x["chat_history"])
    )
    | CONDENSE_QUESTION_PROMPT
    | model
    | StrOutputParser(),
)

_context = RunnablePassthrough.assign(context=itemgetter('standalone_question')|retriever,
                                      question=itemgetter('standalone_question'))

# _context = {"context": itemgetter("standalone_question")|retriever|_combine_documents,
#             "question": lambda x: x["standalone_question"]}

"""
RunnablePassthrough.assign(
    chat_history=lambda x: _format_chat_history(x["chat_history"])
    )
"""

conversational_qa_chain = (
    _inputs | _context | ANSWER_PROMPT | model | StrOutputParser()
)

conversational_qa_chain.invoke({"question": "Translate this sentence from English to French: I love programming.",
                                "chat_history": demo_chat_history})